# Analysis code for the paper "B1+ shimming in the cervical spinal cord at 7T"

## Assumed input data:
This code is written with the assumption that, for every subject, the data is organized as:
_Subject_
__RF shim directory (Seven directories per subject)
    GRE scan
    TFL_B1map scan
__MPRAGE directory

## Analysis procedure:

For each subject:
-->For each RF shim directory

---->Analyze GRE scans by:
----->Segment the SC from the GRE scan
----->Extract the signal intensity within this mask from the GRE scans

---->Analyze B1+ maps by:
----->Coregistering the anatomical image of the TFL_B1map scan to the corresponding GRE scan (noshim to noshim, CVred to CVred, etc)
----->Warp the previously created mask to the TFL_B1map space
----->Convert the B1+ maps to nT/V units
----->Extract the B1+ value within the coregistered mask



In [3]:
#Necessary imports and helper function

import os
import re
import json
import subprocess
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import display
from tabulate import tabulate
import nibabel as nib
import pandas as pd


In [4]:
# Helper functions
import os
import fnmatch


def count_visible_subdirectories(folder_path):
    subdirectories = [subdir for subdir in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, subdir)) and not subdir.startswith('.')]
    return len(subdirectories)

def named_subdir(folder_path,name):
    for dir_name in os.listdir(folder_path):
        if dir_name.lower() == name:
            return os.path.join(folder_path, dir_name)
    return None  # If no matching directory is found


# Helper function to find a specific string in a specific location in a  JSON and return the matching nifti filename
def find_matching_nii_json_pairs(directory_path, keyword, keywordlocation):
    nii_filename = []

    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):
            json_file_path = os.path.join(directory_path, filename)
            nii_file_path = os.path.join(directory_path, filename.replace(".json", ".nii.gz"))

            if os.path.exists(nii_file_path):
                with open(json_file_path, 'r') as json_file:
                    data = json.load(json_file)
                    if keywordlocation in data and keyword in data[keywordlocation]:
                        nii_filename.append(nii_file_path)

    if not nii_filename:
        raise ValueError("No matching nii.gz files found.")

    return nii_filename

def fetch_file_via_name(directory_path,expression):
    directory_abspath=os.path.abspath(directory_path)
    files = [os.path.join(directory_abspath, file) for file in os.listdir(directory_abspath) if os.path.isfile(os.path.join(directory_abspath, file)) and not file.startswith('.')]
    filtered_files = [file for file in files if fnmatch.fnmatch(file, expression)]
    return filtered_files
    
# Helper function for subprocesses
def run_subprocess(cmd):
    """Wrapper for ``subprocess.run()`` that enables to input ``cmd`` as a full string (easier for debugging).
    Args:
        cmd (string): full command to be run on the command line
    """
    try:
        subprocess.run(
            cmd.split(' '),
            text=True,
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
        )
    except subprocess.CalledProcessError as err:
        print(f"Return code: {err.returncode}")
        print("Output:", err.output)
        raise

def GRE_QA_visualizer(datafile,overlayfile):

    datafile_data=nib.load(datafile)
    datafile_data=datafile_data.get_fdata()
    
    overlayfile_data=nib.load(overlayfile)
    overlayfile_data=overlayfile_data.get_fdata()
    overlayfile_data = np.ma.masked_where(overlayfile_data < 0.3, overlayfile_data)
    
    centralslice=(np.floor(datafile_data.shape[0]/2)).astype(int)
    fig, ax = plt.subplots()
    plt.axis('off')
    img1 = ax.imshow(np.rot90(datafile_data[centralslice,:,:]), cmap=plt.cm.gray)
    ax.imshow(np.rot90(overlayfile_data[centralslice,:,:]), cmap=plt.cm.autumn, interpolation='none', alpha=0.5)
    plt.show()

def create_QA_and_Processing_dirs(indir):
    QAdir=os.path.join(indir,'QA')
    if os.path.isdir(QAdir):
        print('The QA directory already exists! Are you rerunnig the script?')
    else:
        os.makedirs(QAdir)
    Processingdir=os.path.join(indir,'Processing')
    if os.path.isdir(Processingdir):
        print('The Processing directory already exists! Are you rerunnig the script?')
    else:
        os.makedirs(Processingdir)
    return QAdir, Processingdir

def create_QA_and_Processing_dirs_B1(indir):
    QAdir=os.path.join(indir,'QA_B1')
    if os.path.isdir(QAdir):
        print('The QA directory already exists! Are you rerunnig the script?')
    else:
        os.makedirs(QAdir)
    Processingdir=os.path.join(indir,'Processing_B1')
    if os.path.isdir(Processingdir):
        print('The Processing directory already exists! Are you rerunnig the script?')
    else:
        os.makedirs(Processingdir)
    return QAdir, Processingdir

def create_QA_and_Processing_dirs_MPRAGE(indir):
    QAdir=os.path.join(indir,'QA_MPRAGE')
    if os.path.isdir(QAdir):
        print('The QA directory already exists! Are you rerunnig the script?')
    else:
        os.makedirs(QAdir)
    Processingdir=os.path.join(indir,'Processing_MPRAGE')
    if os.path.isdir(Processingdir):
        print('The Processing directory already exists! Are you rerunnig the script?')
    else:
        os.makedirs(Processingdir)
    return QAdir, Processingdir

def check_number_of_subdirs(indir,number):
    if not count_visible_subdirectories(indir) ==number:
        print('Unexpected number of subdirectories. Please check your data and rerun!')
        exit()

def check_number_of_GRE_scans_distcorr(filenames):
    if len(filenames)==1 or len(filenames)==2:
        distcorr_filename=filenames[-1] #this is the one with distortion correction
        return distcorr_filename
    else:
        print('Unexpected number of NIFTI files found. Please inspect data and rerun!')
        exit() 

def check_number_of_GRE_scans_nodistcorr(filenames):
    if len(filenames)==1 or len(filenames)==2:
        nodistcorr_filename=filenames[0] #this is the one without distortion correction
        return nodistcorr_filename
    else:
        print('Unexpected number of NIFTI files found. Please inspect data and rerun!')
        exit()

# Extracts the Reference Voltage from a json file
def extract_tx_ref_amp(json_file): #this function will extract the TxRefAmp value of the json file, which is
    # the reference voltage
    with open(json_file, "r") as f:
        data = json.load(f)
        return data.get("TxRefAmp", "N/A")

In [5]:
## Setup for contrast agnostic soft segmentation
## REQUIRES THE USE OF THE VENV_MONAI SET UP BELOW
## Requires the notebook being run with the venv_monai kernel!
#1 Install the necessary tools as described here:
# https://github.com/sct-pipeline/contrast-agnostic-softseg-spinalcord/tree/main/monai
#2, Download the inference dataset (model 2023 09 18 zip) from here:
# https://github.com/sct-pipeline/contrast-agnostic-softseg-spinalcord/releases/tag/v2.0
# And unzip it into a folder
monai_checkpoint='/Users/danielpapp/model_2023-09-18/'

#and create an alias for calling it, otherwise its going to be a lot of typing
monai_command='python /Users/danielpapp/contrast-agnostic-softseg-spinalcord-2.0/monai/run_inference_single_image.py'

# reinstalling requrements, just in case
!pip install -r /Users/danielpapp/contrast-agnostic-softseg-spinalcord-2.0/monai/requirements_inference_cpu.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu


In [6]:
def GRE_segment_monai(subjectpath, indirname):
    #Define the GRE scan directory the GRE scans
    GRE_dir=os.path.join(subjectpath,'GRE_B1')   

    #We start by finding the directory, and creating the subdirectories for processing and QA
    dirname_shimcase=named_subdir(GRE_dir,indirname)
    [QAdir, Processingdir]=create_QA_and_Processing_dirs(dirname_shimcase)
    
    #We fetch the GRE scans and check that there are only one or two (Wih and without distortion correction)
    #Changed to use the one WITHOUT distortion correction, as this will be needed for B1 processing
    GRE_filenames=fetch_file_via_name(dirname_shimcase,'*gre*nii.gz')
    GRE_shimfile_nodistcorr=check_number_of_GRE_scans_nodistcorr(GRE_filenames)
    
    #Sometimes there is a 0 -0 discrepancy between the QFORM and SFORM for MGH data. To fix this, we use the 
    # 'set-qform-to-sfrom' option of sct_image
    run_subprocess(f"sct_image -i {GRE_shimfile_nodistcorr} -set-qform-to-sform")
    
    #We can now segment this file
    run_subprocess(f"{monai_command} --path-img {GRE_shimfile_nodistcorr} --path-out {Processingdir} --chkp-path {monai_checkpoint}")
    
    #And turn the soft segmentation into a hard one, so as to not mix evaluating the segmentation and the RF shimming
    GRE_shim_monaisegfilename=fetch_file_via_name(Processingdir,'*gre*_pred.nii.gz')
    GRE_shim_monaisegfilename=GRE_shim_monaisegfilename[0]
    GRE_shim_monaisegfilename_hard=GRE_shim_monaisegfilename.split('.')[0]+'_hard.nii.gz'
    run_subprocess(f"sct_maths -i {GRE_shim_monaisegfilename} -o {GRE_shim_monaisegfilename_hard} -bin 0.5")
    
    #And then we need to do manual checking
    #And extract the signal intensity    
    #GRE_shim_segfilename=fetch_file_via_name(Processingdir,'*gre*_seg.nii.gz')
    #GRE_shim_segfilename=GRE_shim_segfilename[0]
    #GRE_shim_CSVfile=os.path.join(Processingdir,GRE_shimfile_distcorr.split('/')[-1].split('.')[0]+'_sigint.csv')
    #run_subprocess(f"sct_extract_metric -i {GRE_shimfile_distcorr} -f {GRE_shim_segfilename} -o {GRE_shim_CSVfile} -perslice 1")


In [7]:
def GRE_extract_monai(subjectpath,indirname):
    GRE_dir=os.path.join(subjectpath,'GRE_B1')   

    #We start by finding the directory, and creating the subdirectories for processing and QA
    dirname_shimcase=named_subdir(GRE_dir,indirname)
    [QAdir, Processingdir]=create_QA_and_Processing_dirs(dirname_shimcase)
    
    #We fetch the GRE scans and check that there are only one or two (Wih and without distortion correction)
    #Changed to use the one WITHOUT distortion correction, as this will be needed for B1 processing
    GRE_filenames=fetch_file_via_name(dirname_shimcase,'*gre*nii.gz')
    GRE_shimfile_nodistcorr=check_number_of_GRE_scans_nodistcorr(GRE_filenames)
    
    #And extract the signal intensity
    #Check if there is a corrected file
    if not fetch_file_via_name(Processingdir,'*gre*_pred_corr.nii.gz'):
        GRE_shim_monaisegfilename=fetch_file_via_name(Processingdir,'*gre*_pred_hard.nii.gz')
    else:
        GRE_shim_monaisegfilename=fetch_file_via_name(Processingdir,'*gre*_pred_hard_corr.nii.gz')
    GRE_shim_monaisegfilename=GRE_shim_monaisegfilename[0]
    
    GRE_shim_CSVfile=os.path.join(Processingdir,GRE_shimfile_nodistcorr.split('/')[-1].split('.')[0]+'_sigint.csv')
    run_subprocess(f"sct_extract_metric -i {GRE_shimfile_nodistcorr} -f {GRE_shim_monaisegfilename} -o {GRE_shim_CSVfile} -perslice 1")


    #And extract the signal intensity    
    #GRE_shim_segfilename=fetch_file_via_name(Processingdir,'*gre*_seg.nii.gz')
    #GRE_shim_segfilename=GRE_shim_segfilename[0]
    #GRE_shim_CSVfile=os.path.join(Processingdir,GRE_shimfile_distcorr.split('/')[-1].split('.')[0]+'_sigint.csv')
    #run_subprocess(f"sct_extract_metric -i {GRE_shimfile_distcorr} -f {GRE_shim_segfilename} -o {GRE_shim_CSVfile} -perslice 1")


In [8]:
def B1_map_process_monai(subjectpath,indirname):
    #Define the GRE scan directory the GRE scans
    GRE_dir=os.path.join(subjectpath,'GRE_B1')   

    #We start by finding the directory, and creating the subdirectories for processing and QA
    dirname_shimcase=named_subdir(GRE_dir,indirname)
    [QAdir_B1, Processingdir_B1]=create_QA_and_Processing_dirs_B1(dirname_shimcase)
    [QAdir, Processingdir]=create_QA_and_Processing_dirs(dirname_shimcase)
   
    #Find GRE filenames
    GRE_filenames=fetch_file_via_name(dirname_shimcase,'*gre*nii.gz')
    GRE_shimfile_nodistcorr=check_number_of_GRE_scans_nodistcorr(GRE_filenames)
    
    #We fetch the proper segmentation
    if not fetch_file_via_name(Processingdir,'*gre*_pred_corr.nii.gz'):
        GRE_shim_monaisegfilename=fetch_file_via_name(Processingdir,'*gre*_pred_hard.nii.gz')
    else:
        GRE_shim_monaisegfilename=fetch_file_via_name(Processingdir,'*gre*_pred_hard_corr.nii.gz')
    GRE_shim_monaisegfilename=GRE_shim_monaisegfilename[0]
    
    #Next we find the TFL_B1maps anatomical and flip angle maps
    TFL_anatfile=find_matching_nii_json_pairs(dirname_shimcase, 'anatomical', 'ImageComments')[0]
    TFL_FAfile=find_matching_nii_json_pairs(dirname_shimcase, 'angle map', 'ImageComments')[0]

    #We then convert the flip angle map into a nT/V map
    # Maths from Kyle Gilbert
    #GAMMA = 2.675e8
    #B1eff_mag = (AcquiredFA ./ RequestedFA) .* (pi ./ (GAMMA .* 1e-3 .* VoltageAtSocket)); % [T/V]
    #B1eff_mag = B1eff_mag .* 1e9; % [T/V] to [nT/V]
    # The costants sum up to 130.492, so to convert the B1map to nT/V, it has to be divided by 10 (to get it back into units of FA)
    # then multiplied by 130.492 and divided by the VoltageAtSocket

    TFL_FAfile_nTpV=os.path.join(Processingdir_B1,TFL_FAfile.split('.')[0].split('/')[-1]+'_nTpV.nii.gz')
    TFL_FAfile_jsonfile = TFL_FAfile.replace(".nii.gz", ".json")
    RefVol = extract_tx_ref_amp(TFL_FAfile_jsonfile)
    VoltageAtSocket = RefVol * 10**-0.095
    VoltageAtSocket=np.around(VoltageAtSocket, decimals=2)
    run_subprocess(f"sct_maths -i {TFL_FAfile} -div 10 -o {TFL_FAfile_nTpV}")
    run_subprocess(f"sct_maths -i {TFL_FAfile_nTpV} -div {VoltageAtSocket} -o {TFL_FAfile_nTpV}")
    run_subprocess(f"sct_maths -i {TFL_FAfile_nTpV} -mul 130.492 -o {TFL_FAfile_nTpV}")

    #Then we coreigster the B1map anatomical to the undistcorrected GRE
    warp_anat_2_B1_fname=os.path.join(Processingdir_B1,GRE_shimfile_nodistcorr.split('.')[0].split('/')[-1]+'_warp2B1.nii.gz')
    run_subprocess(f"sct_register_multimodal -i {TFL_anatfile} -d {GRE_shimfile_nodistcorr} -ofolder {Processingdir_B1} -qc {QAdir_B1} -dseg {GRE_shim_monaisegfilename} -owarp {warp_anat_2_B1_fname}")


    #Apply this transformation to the segmentation we just produced
    warped_segname=os.path.join(Processingdir_B1,GRE_shim_monaisegfilename.split('.')[0].split('/')[-1]+'_warped.nii.gz')
    run_subprocess(f"sct_apply_transfo -i {GRE_shim_monaisegfilename} -d {TFL_anatfile} -o {warped_segname} -x nn -w {warp_anat_2_B1_fname}")

    #And finally extact the metric
    TFL_FAfile_nTpV_CSV=(TFL_FAfile_nTpV.split('.')[0]+'_SC.csv')
    run_subprocess(f"sct_extract_metric -i {TFL_FAfile_nTpV} -f {warped_segname} -o {TFL_FAfile_nTpV_CSV} -perslice 1")

In [9]:
def MPRAGE_segment_monai(subjectpath):
    #Define the GRE scan directory the GRE scans
    MPRAGE_dir=os.path.join(subjectpath,'MPRAGE')   

    #We start by finding the directory, and creating the subdirectories for processing and QA
    [QAdir, Processingdir]=create_QA_and_Processing_dirs(MPRAGE_dir)
    
    MPRAGE_filenames_noshim=fetch_file_via_name(MPRAGE_dir,'*mprage*no*nii.gz')
    MPRAGE_shimfile_noshim=check_number_of_GRE_scans_distcorr(MPRAGE_filenames_noshim)

    MPRAGE_filenames_rfshim=fetch_file_via_name(MPRAGE_dir,'*mprage*CV*nii.gz')
    MPRAGE_shimfile_rfshim=check_number_of_GRE_scans_distcorr(MPRAGE_filenames_rfshim)
    
    #Sometimes there is a 0 -0 discrepancy between the QFORM and SFORM for MGH data. To fix this, we use the 
    # 'set-qform-to-sfrom' option of sct_image
    #run_subprocess(f"sct_image -i {MPRAGE_shimfile_distcorr} -set-qform-to-sform")
    
    #We can now segment this files
    run_subprocess(f"{monai_command} --path-img {MPRAGE_shimfile_noshim} --path-out {Processingdir} --chkp-path {monai_checkpoint}")
    run_subprocess(f"{monai_command} --path-img {MPRAGE_shimfile_rfshim} --path-out {Processingdir} --chkp-path {monai_checkpoint}")
    
    #And turn the soft segmentation into a hard one, so as to not mix evaluating the segmentation and the RF shimming
    MPRAGE_shim_monaisegfilename_noshim=fetch_file_via_name(Processingdir,'*mprage*no*_pred.nii.gz')
    MPRAGE_shim_monaisegfilename_noshim=MPRAGE_shim_monaisegfilename_noshim[0]
    MPRAGE_shim_monaisegfilename_hard_noshim=MPRAGE_shim_monaisegfilename_noshim.split('.')[0]+'_hard.nii.gz'
    run_subprocess(f"sct_maths -i {MPRAGE_shim_monaisegfilename_noshim} -o {MPRAGE_shim_monaisegfilename_hard_noshim} -bin 0.5")
    
    #And turn the soft segmentation into a hard one, so as to not mix evaluating the segmentation and the RF shimming
    MPRAGE_shim_monaisegfilename_rfshim=fetch_file_via_name(Processingdir,'*mprage*CV*_pred.nii.gz')
    MPRAGE_shim_monaisegfilename_rfshim=MPRAGE_shim_monaisegfilename_rfshim[0]
    MPRAGE_shim_monaisegfilename_hard_rfshim=MPRAGE_shim_monaisegfilename_rfshim.split('.')[0]+'_hard.nii.gz'
    run_subprocess(f"sct_maths -i {MPRAGE_shim_monaisegfilename_rfshim} -o {MPRAGE_shim_monaisegfilename_hard_rfshim} -bin 0.5")
    
    #And then we need to do manual checking
    #And extract the signal intensity    
    #GRE_shim_segfilename=fetch_file_via_name(Processingdir,'*gre*_seg.nii.gz')
    #GRE_shim_segfilename=GRE_shim_segfilename[0]
    #GRE_shim_CSVfile=os.path.join(Processingdir,GRE_shimfile_distcorr.split('/')[-1].split('.')[0]+'_sigint.csv')
    #run_subprocess(f"sct_extract_metric -i {GRE_shimfile_distcorr} -f {GRE_shim_segfilename} -o {GRE_shim_CSVfile} -perslice 1")


In [10]:
def MPRAGE_extract_monai(subjectpath):
    MPRAGE_dir=os.path.join(subjectpath,'MPRAGE')   

    #We start by finding the directory, and creating the subdirectories for processing and QA
    dirname_shimcase=named_subdir(MPRAGE_dir)
    [QAdir, Processingdir]=create_QA_and_Processing_dirs_MPRAGE(dirname_shimcase)
    
    #Fetch filenames
    MPRAGE_filenames=fetch_file_via_name(dirname_shimcase,'*mprage*no*nii.gz')
    MPRAGE_shimfile_noshim=check_number_of_GRE_scans_distcorr(MPRAGE_filenames)
    
    MPRAGE_filenames=fetch_file_via_name(dirname_shimcase,'*mprage*rf*nii.gz')
    MPRAGE_shimfile_rfshim=check_number_of_GRE_scans_distcorr(MPRAGE_filenames)
    #And extract the signal intensity
    #Check if there is a corrected file
    if not fetch_file_via_name(Processingdir,'*mprage*no*_pred_corr.nii.gz'):
        MPRAGE_shim_monaisegfilename_noshim=fetch_file_via_name(Processingdir,'*mprage*no*_pred.nii.gz')
    else:
        MPRAGE_shim_monaisegfilename_noshim=fetch_file_via_name(Processingdir,'*mprage*no*_pred_corr.nii.gz')
    MPRAGE_shim_monaisegfilename_noshim=MPRAGE_shim_monaisegfilename_noshim[0]
    
    if not fetch_file_via_name(Processingdir,'*mprage*rf*_pred_corr.nii.gz'):
        MPRAGE_shim_monaisegfilename_rfshim=fetch_file_via_name(Processingdir,'*mprage*rf*_pred.nii.gz')
    else:
        MPRAGE_shim_monaisegfilename_rfshim=fetch_file_via_name(Processingdir,'*mprage*rf*_pred_corr.nii.gz')
    MPRAGE_shim_monaisegfilename_rfshim=MPRAGE_shim_monaisegfilename_rfshim[0]
    
    MPRAGE_noshim_CSVfile=os.path.join(Processingdir,MPRAGE_shimfile_noshim.split('/')[-1].split('.')[0]+'_sigint.csv')
    run_subprocess(f"sct_extract_metric -i {MPRAGE_shimfile_noshim} -f {MPRAGE_shim_monaisegfilename_noshim} -o {MPRAGE_noshim_CSVfile} -perslice 1")
   
    MPRAGE_rfshim_CSVfile=os.path.join(Processingdir,MPRAGE_shimfile_rfshim.split('/')[-1].split('.')[0]+'_sigint.csv')
    run_subprocess(f"sct_extract_metric -i {MPRAGE_shimfile_rfshim} -f {MPRAGE_shim_monaisegfilename_rfshim} -o {MPRAGE_rfshim_CSVfile} -perslice 1")



In [37]:
# Running MONAI GRE segmentation subject by subject to allow for  manual correction
subjectpath='/Users/danielpapp/DATA/RF_shimming_project_monai/SubE'
shimcasenames=['noshim','patspec','volspec','phaseonly','cvred','target','sareff']
for shimcase in range(len(shimcasenames)):
    GRE_segment_monai(subjectpath,shimcasenames[shimcase])
    print(shimcase)
    #B1_map_process(subjectpath,shimcasenames[shimcase])

0
1
2
3
4
5
6


In [ ]:
#Manual correction was necessary for the following cases:
#SubA all OK

#SubB CVred
#SubB PhaseOnly
#SubB Target

#SubC CVred
#SubC PhaseOnly

#SubD CVred
#SubD PatSpec
#SubD PhaseOnly

#Manual correction has to still be performed for:


#OVERALL LOW CONFIDENCE IN SUB D MANUAL CORRECTIONS

In [60]:
# Running MONAI GRE signal extraction subject by subject 
subjectpath='/Users/danielpapp/DATA/RF_shimming_project_monai/SubC'
shimcasenames=['noshim','patspec','volspec','phaseonly','cvred','target','sareff']
for shimcase in range(len(shimcasenames)):
    GRE_extract_monai(subjectpath,shimcasenames[shimcase])
    print(shimcase)
    #B1_map_process(subjectpath,shimcasenames[shimcase])

The QA directory already exists! Are you rerunnig the script?
The Processing directory already exists! Are you rerunnig the script?
0
The QA directory already exists! Are you rerunnig the script?
The Processing directory already exists! Are you rerunnig the script?
1
The QA directory already exists! Are you rerunnig the script?
The Processing directory already exists! Are you rerunnig the script?
2
The QA directory already exists! Are you rerunnig the script?
The Processing directory already exists! Are you rerunnig the script?
3
The QA directory already exists! Are you rerunnig the script?
The Processing directory already exists! Are you rerunnig the script?
4
The QA directory already exists! Are you rerunnig the script?
The Processing directory already exists! Are you rerunnig the script?
5
The QA directory already exists! Are you rerunnig the script?
The Processing directory already exists! Are you rerunnig the script?
6


In [13]:
# Running MONAI MPRAGE segmentation subject by subject 
subjectpath='/Users/danielpapp/DATA/RF_shimming_project_monai/SubC'
MPRAGE_segment_monai(subjectpath)


In [77]:
# THIS TOTALLY FAILED ON SUB A NOSHIM
# TODO CHECK WHAT THE HELL IS GOING ON And processd

# Running MONAI B1 processing subject by subject 
subjectpath='/Users/danielpapp/DATA/RF_shimming_project_monai/SubA'
shimcasenames=['noshim','patspec','volspec','phaseonly','cvred','target','sareff']
for shimcase in range(len(shimcasenames)):
    B1_map_process_monai(subjectpath,shimcasenames[shimcase])
    print(shimcase)
    #B1_map_process(subjectpath,shimcasenames[shimcase])

The QA directory already exists! Are you rerunnig the script?
The Processing directory already exists! Are you rerunnig the script?


KeyboardInterrupt: 

In [ ]:
########## OLD CODE STARTS HERE

In [25]:
def GRE_segment_extract(subjectpath, indirname):
    #Define the GRE scan directory the GRE scans
    GRE_dir=os.path.join(subjectpath,'GRE_B1')   

    #We start by finding the directory, and creating the subdirectories for processing and QA
    dirname_shimcase=named_subdir(GRE_dir,indirname)
    [QAdir, Processingdir]=create_QA_and_Processing_dirs(dirname_shimcase)
    
    #We fetch the GRE scans and check that there are only one or two (Wih and without distortion correction)
    GRE_filenames=fetch_file_via_name(dirname_shimcase,'*gre*nii.gz')
    GRE_shimfile_distcorr=check_number_of_GRE_scans_distcorr(GRE_filenames)
    
    #Sometimes there is a 0 -0 discrepancy between the QFORM and SFORM for MGH data. To fix this, we use the 
    # 'set-qform-to-sfrom' option of sct_image
    run_subprocess(f"sct_image -i {GRE_shimfile_distcorr} -set-qform-to-sform")
    
    #We can now segment this file
    run_subprocess(f"sct_deepseg_sc -i {GRE_shimfile_distcorr} -c t2s -kernel 2d -centerline svm -ofolder {Processingdir} -qc {QAdir}")

    #And extract the signal intensity    
    GRE_shim_segfilename=fetch_file_via_name(Processingdir,'*gre*_seg.nii.gz')
    GRE_shim_segfilename=GRE_shim_segfilename[0]
    GRE_shim_CSVfile=os.path.join(Processingdir,GRE_shimfile_distcorr.split('/')[-1].split('.')[0]+'_sigint.csv')
    run_subprocess(f"sct_extract_metric -i {GRE_shimfile_distcorr} -f {GRE_shim_segfilename} -o {GRE_shim_CSVfile} -perslice 1")


In [24]:
def B1_map_process(subjectpath,indirname):
    #Define the GRE scan directory the GRE scans
    GRE_dir=os.path.join(subjectpath,'GRE_B1')   

    #We start by finding the directory, and creating the subdirectories for processing and QA
    dirname_shimcase=named_subdir(GRE_dir,indirname)
    [QAdir_B1, Processingdir_B1]=create_QA_and_Processing_dirs_B1(dirname_shimcase)
    
    #Next we find the TFL_B1maps anatomical and flip angle maps
    TFL_anatfile=find_matching_nii_json_pairs(dirname_shimcase, 'anatomical', 'ImageComments')[0]
    TFL_FAfile=find_matching_nii_json_pairs(dirname_shimcase, 'angle map', 'ImageComments')[0]

    #We then convert the flip angle map into a nT/V map
    # Maths from Kyle Gilbert
    #GAMMA = 2.675e8
    #B1eff_mag = (AcquiredFA ./ RequestedFA) .* (pi ./ (GAMMA .* 1e-3 .* VoltageAtSocket)); % [T/V]
    #B1eff_mag = B1eff_mag .* 1e9; % [T/V] to [nT/V]
    # The costants sum up to 130.492, so to convert the B1map to nT/V, it has to be divided by 10 (to get it back into units of FA)
    # then multiplied by 130.492 and divided by the VoltageAtSocket

    TFL_FAfile_nTpV=os.path.join(Processingdir_B1,TFL_FAfile.split('.')[0].split('/')[-1]+'_nTpV.nii.gz')
    TFL_FAfile_jsonfile = TFL_FAfile.replace(".nii.gz", ".json")
    RefVol = extract_tx_ref_amp(TFL_FAfile_jsonfile)
    VoltageAtSocket = RefVol * 10**-0.095
    VoltageAtSocket=np.around(VoltageAtSocket, decimals=2)
    run_subprocess(f"sct_maths -i {TFL_FAfile} -div 10 -o {TFL_FAfile_nTpV}")
    run_subprocess(f"sct_maths -i {TFL_FAfile_nTpV} -div {VoltageAtSocket} -o {TFL_FAfile_nTpV}")
    run_subprocess(f"sct_maths -i {TFL_FAfile_nTpV} -mul 130.492 -o {TFL_FAfile_nTpV}")

    #Next we fing the distoriton uncorrected GRE scan, since it is easier to coregister that to the also distortion uncorrected B1 map
    GRE_filenames=fetch_file_via_name(dirname_shimcase,'*gre*nii.gz')
    GRE_shimfile_nodistcorr=check_number_of_GRE_scans_nodistcorr(GRE_filenames)

    #Sometimes there is a 0 -0 discrepancy between the QFORM and SFORM for MGH data. To fix this, we use the 
    # 'set-qform-to-sfrom' option of sct_image
    run_subprocess(f"sct_image -i {GRE_shimfile_nodistcorr} -set-qform-to-sform")
    
    #We can now segment this file
    run_subprocess(f"sct_deepseg_sc -i {GRE_shimfile_nodistcorr} -c t2s -kernel 2d -centerline svm -ofolder {Processingdir_B1} -qc {QAdir_B1}")
    GRE_shim_segfilename=fetch_file_via_name(Processingdir_B1,'*gre*_seg.nii.gz')
    GRE_shim_segfilename=GRE_shim_segfilename[0]

    #Then we coreigster the B1map anatomical to the undistcorrected GRE
    warp_anat_2_B1_fname=os.path.join(Processingdir_B1,GRE_shimfile_nodistcorr.split('.')[0].split('/')[-1]+'_warp2B1.nii.gz')
    run_subprocess(f"sct_register_multimodal -i {TFL_anatfile} -d {GRE_shimfile_nodistcorr} -ofolder {Processingdir_B1} -qc {QAdir_B1} -dseg {GRE_shim_segfilename} -owarp {warp_anat_2_B1_fname}")

    #Apply this transformation to the segmentation we just produced
    warped_segname=os.path.join(Processingdir_B1,GRE_shim_segfilename.split('.')[0].split('/')[-1]+'_warped.nii.gz')
    run_subprocess(f"sct_apply_transfo -i {GRE_shim_segfilename} -d {TFL_anatfile} -o {warped_segname} -x nn -w {warp_anat_2_B1_fname}")

    #And finally extact the metric
    TFL_FAfile_nTpV_CSV=(TFL_FAfile_nTpV.split('.')[0]+'_SC.csv')
    run_subprocess(f"sct_extract_metric -i {TFL_FAfile_nTpV} -f {warped_segname} -o {TFL_FAfile_nTpV_CSV} -perslice 1")

In [15]:
subjectpath='/Users/danielpapp/DATA/RF_shimming_project_clean/SubA'
shimcasenames=['noshim','patspec','volspec','phaseonly','cvred','target','sareff']
for shimcase in range(len(shimcasenames)):
    GRE_segment_extract(subjectpath,shimcasenames[shimcase])
    B1_map_process(subjectpath,shimcasenames[shimcase])


In [16]:
subjectpath='/Users/danielpapp/DATA/RF_shimming_project_clean/SubB'
shimcasenames=['noshim','patspec','volspec','phaseonly','cvred','target','sareff']
for shimcase in range(len(shimcasenames)):
    GRE_segment_extract(subjectpath,shimcasenames[shimcase])
    B1_map_process(subjectpath,shimcasenames[shimcase])